In [ ]:
# repo_dir = "Repos"   # Set this to be where your github repos are located.
# %load_ext autoreload
# %autoreload 2

# # Update the load path so python can find modules for the model
# import sys
# from pathlib import Path
# sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
# sys.path.insert(0, str(Path.home() / repo_dir / "deriva-ml"))

In [ ]:
# Prerequisites
from eye_ai.eye_ai import EyeAI
from deriva_ml.dataset_bag import DatasetBag

from pathlib import Path, PurePath

from deriva_ml.deriva_ml_base import MLVocab as vc
from deriva_ml.execution_configuration import ExecutionConfiguration, Workflow, Execution

In [ ]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
# host = 'www.eye-ai.org'
host = 'dev.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [ ]:
cache_dir = Path.home() / '/data'
working_dir = Path.home() / '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)
pb = EA.catalog.getPathBuilder()

# Configuration

In [ ]:
# Add Workflow Vocab terms
EA.add_term(vc.workflow_type, "Test Workflow", description="A test Workflow for new DM")
# Workflow instance
test_workflow = Workflow(
    name="Dataset creation template",
    url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/templates/template_dataset.ipynb",
    workflow_type="Test Workflow"
)
# Configuration instance
config = ExecutionConfiguration(
    bdbags=['2-7K8W'], #2-7K8W
    models = ['2-4JR6'],
    execution=Execution(description="Sample Test Execution"),
    workflow=test_workflow,
    description="Our Test Workflow instance")
# Initialize execution
configuration_record = EA.initialize_execution(config)
execution_rid = configuration_record.execution_rid

In [ ]:
print(configuration_record)

# Create DatasetBag

In [ ]:
ds_bag = DatasetBag(configuration_record.bag_paths[0])

In [ ]:
training_rids = #slice the dataset and extract a list of subject rid

# Create dataset

In [ ]:
new_dataset_rid = EA.create_dataset(['LAC', 'Training'], description='A test dataset')

# Add subjects into the new dataset

In [ ]:
EA.add_dataset_members(dataset_rid=new_dataset_rid, members=training_rids)

# Add subdatasets to a dataset

In [ ]:
subsets = [] # A list of (sub)dataset rids
EA.add_dataset_members(dataset_rid=new_dataset_rid, members=subsets)

# Upload results

In [ ]:
# crete asset path
asset_type_name = ""
asset_path = configuration_record.execution_assets_path(asset_type_name)
# save assets to asset_path


# upload assets to catalog
EA.upload_execution(configuration_record, True)